# Code to compare

In [ ]:
import pandas as pd
import json
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gc

In [ ]:
MODEL_CHOICE = "TinyLlama"  # Changing this to test different models

# Available models:
MODELS = {
    "Dolly-v2-12B": "databricks/dolly-v2-12b",
    "Phi-3": "microsoft/Phi-3-mini-4k-instruct",
    "Qwen-7B": "Qwen/Qwen2.5-7B-Instruct",
    "Gemma-2B": "google/gemma-2b-it",
    "TinyLlama": "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
}

MODEL_NAME = MODELS[MODEL_CHOICE]

print(f"\n Testing on {MODEL_CHOICE} ({MODEL_NAME})")


In [ ]:
# To see if there is GPU
if not torch.cuda.is_available():
    print("No GPU!")
    exit()
else:
    print(f"GPU: {torch.cuda.get_device_name(0)}")\
    


In [ ]:
# Files
FILES = {
    'Hindi': 'Examples for annotations - Hindi.csv',
    'Dutch': 'Examples for annotations - Dutch.csv',
    'Romanian': 'Examples for annotations - Romanian.csv'
}

EMOTIONS = ['Joy', 'Sadness', 'Anger', 'Fear', 'Disgust', 'Surprise', 'NULL']

# Loadind the model

try:
    # 4-bit quantization to save memory
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
    )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True
    )

    print(f"✅ {MODEL_CHOICE} loaded successfully!")

except Exception as e:
    print(f"Error loading model: {str(e)[:200]}")
    if "token" in str(e).lower() or "gated" in str(e).lower():
        print("\nNeed to go to HuggingFace to load the key")
    exit()

In [ ]:
# Function to test the prediction
def predict_emotion(text, language):

    # Create simple prompt
    prompt = f"""Analyze this text and identify the PRIMARY emotion.

Text: "{text[:300]}"
Language: {language}

Choose ONE emotion: Joy, Sadness, Anger, Fear, Disgust, Surprise, or NULL

Emotion:"""

    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=600)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=10,
                temperature=0.1,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

        # Extract emotion
        for emotion in EMOTIONS:
            if emotion.lower() in response.lower():
                return emotion

        # Return first word if no match
        return response.strip().split()[0] if response.strip() else "NULL"

    except Exception as e:
        print(f"Error: {str(e)[:50]}")
        return "ERROR"

In [ ]:
# Test on all 3 languages

all_results = []

for lang, filepath in FILES.items():
    print(f"\n Testing {lang}...")

    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
        continue

    # Load data
    df = pd.read_csv(filepath)
    emotion_col = 'Annotated Emotions' if 'Annotated Emotions' in df.columns else 'Annotated Emotions 1'

    # Get examples with emotions
    df_test = df[df[emotion_col].notna() & (df[emotion_col] != 'NULL')]

    if len(df_test) == 0:
        print(f"No examples found")
        continue

    # Test each example
    correct = 0
    results = []

    for idx, row in df_test.iterrows():
        text = str(row['Text'])
        true_emotion = str(row[emotion_col])

        print(f"   Testing {len(results)+1}/10...", end='\r')

        predicted = predict_emotion(text, lang)

        is_correct = (predicted == true_emotion)
        if is_correct:
            correct += 1

        results.append({
            'text': text[:60],
            'true': true_emotion,
            'predicted': predicted,
            'correct': is_correct
        })

    accuracy = (correct / len(results) * 100) if results else 0

    print(f"\n   ✅ {lang}: {accuracy:.1f}% ({correct}/10)")

    # Show a few examples
    print(f"Sample predictions:")
    for i, r in enumerate(results[:3], 1):
        status = "✅" if r['correct'] else "❌"
        print(f"      {status} True: {r['true']:10s} | Pred: {r['predicted']:10s}")

    all_results.append({
        'model': MODEL_CHOICE,
        'language': lang,
        'accuracy': accuracy,
        'correct': correct,
        'total': len(results),
        'results': results
    })

In [ ]:
# Save results

if all_results:
    # Save JSON
    output_file = f"{MODEL_CHOICE.replace('.', '_')}_results.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)

    # Create summary
    summary_df = pd.DataFrame([{
        'Model': r['model'],
        'Language': r['language'],
        'Accuracy': f"{r['accuracy']:.1f}%",
        'Correct': r['correct'],
        'Total': r['total']
    } for r in all_results])

    csv_file = f"{MODEL_CHOICE.replace('.', '_')}_summary.csv"
    summary_df.to_csv(csv_file, index=False)

    print(f"Saved: {output_file}")
    print(f"Saved: {csv_file}")

    # Summary
    print("\n SUMMARY:")
    print(summary_df.to_string(index=False))

    overall_correct = sum(r['correct'] for r in all_results)
    overall_total = sum(r['total'] for r in all_results)
    overall_accuracy = (overall_correct / overall_total * 100) if overall_total > 0 else 0

    print(f"\n OVERALL: {overall_accuracy:.1f}% ({overall_correct}/{overall_total})")

    # Download files
    try:
        from google.colab import files
        print("\n Downloading results...")
        files.download(csv_file)
        files.download(output_file)
        print(" Downloaded!")
    except:
        print("\n Download manually from file browser ←")

else:
    print("No results to save")